In [111]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [112]:
df = pd.read_csv('data_scrape.csv')
df = df[df["Tên kênh"].isin(["Wowy", "Đen Vâu Official", "Pháo Northside"])]
df = df.reset_index(drop = True)

label_encoder = LabelEncoder()

# Mã hóa tên kênh
df['Tên kênh Label'] = label_encoder.fit_transform(df['Tên kênh'])
df = df.drop(columns=["Tiêu đề", "Thời lượng video", "Ngày phát hành", "Tag", "Tên kênh"])
df = df.dropna()


In [113]:
# Chia thành tập 8:1:1 - (train,validation,test)
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Chia thành tập train + temp
for train_index, temp_index in split.split(df, df['Tên kênh Label']):
    train = df.iloc[train_index]
    temp = df.iloc[temp_index]

# StratifiedShuffleSplit
split_temp = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)

for validation_index, test_index in split_temp.split(temp, temp['Tên kênh Label']):
    validation = temp.iloc[validation_index]
    test = temp.iloc[test_index]

print(f"Tập train: {train.shape}")
print(f"Tập validation: {validation.shape}")
print(f"Tập test: {test.shape}")

Tập train: (276, 4)
Tập validation: (34, 4)
Tập test: (35, 4)


In [114]:
train = train.reset_index(drop=True)
validation = validation.reset_index(drop=True)
test = test.reset_index(drop=True)

In [115]:
# Chuẩn hóa
scaler = StandardScaler()

# Cột cần chuẩn hóa
columns_to_scale = [col for col in train.select_dtypes(include=['float64', 'int64']).columns if col != 'Tên kênh Label']

train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])
validation[columns_to_scale] = scaler.transform(validation[columns_to_scale])
test[columns_to_scale] = scaler.transform(test[columns_to_scale])

print("Tập train sau chuẩn hóa:")
print(train.head())
print("Tập validation sau chuẩn hóa:")
print(validation.head())
print("Tập test sau chuẩn hóa:")
print(test.head())


Tập train sau chuẩn hóa:
   Số lượt thích  Số lượt xem  Số bình luận  Tên kênh Label
0      -0.321173    -0.295194     -0.322805               2
1       0.196715    -0.166540      0.323527               1
2      -0.216945    -0.286220     -0.305898               2
3      -0.307554    -0.282211     -0.306205               2
4      -0.308680    -0.291751     -0.300057               1
Tập validation sau chuẩn hóa:
   Số lượt thích  Số lượt xem  Số bình luận  Tên kênh Label
0      -0.265471    -0.282273     -0.272236               2
1       0.062015    -0.201439      0.175508               1
2      -0.031739    -0.036728     -0.250410               0
3      -0.322221    -0.294137     -0.314813               1
4      -0.318221    -0.287025     -0.313429               2
Tập test sau chuẩn hóa:
   Số lượt thích  Số lượt xem  Số bình luận  Tên kênh Label
0      -0.325474    -0.293389     -0.318655               1
1      -0.330856    -0.296592     -0.325418               1
2       6.877335     

In [116]:
# Tách đặc trưng và nhãn
X_train = train.drop(columns=['Tên kênh Label'])
y_train = train['Tên kênh Label']

X_validation = validation.drop(columns=['Tên kênh Label'])
y_validation = validation['Tên kênh Label']

X_test = test.drop(columns=['Tên kênh Label'])
y_test = test['Tên kênh Label']


In [117]:
# Tạo mô hình SVM với kernel RBF
svm = SVC(kernel='rbf', random_state=42)

# Huấn luyện mô hình
svm.fit(X_train, y_train)

SVC(random_state=42)

In [118]:
# Test trên tập validation
y_val_pred = svm.predict(X_validation)
val_accuracy = accuracy_score(y_validation, y_val_pred)
print(f"Độ chính xác trên tập validation: {val_accuracy:.4f}")

# Test trên tập test
y_test_pred = svm.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Độ chính xác trên tập test: {test_accuracy:.4f}")


Độ chính xác trên tập validation: 0.5882
Độ chính xác trên tập test: 0.6857
